# Administracion de memoria

## Los primeros sistemas 

El sistema operativo tan solo pesaba 64KB y el resto estaba disponible para un unico programa. 

<img src="Imagenes/ad_mem_early_days.png">

## Multiprogramacion y time sharing

Aqui la computacion avanza y las computadoras comienzan a tener mas de proceso, los cuales se guardaban de la suguiente manera:

<img src="Imagenes/admin_mem_multiprogramacion.png">

Notar que la memoria de un proceso se guarda toda junta, el codigo, los datos, etc.

## El espacio de direcciones o address space

El <b>Address Space</b> de un <b>proceso</b> contiene todo el estado de la memoria de un <b>programa en ejecucion</b>.

<img src="Imagenes/admin_mem_address_space.png">

Secciones:
- <b>Program code</b>: Es el codigo del programa. Arranca en la direccion 0.
- <b>Heap</b>: Heap de proceso. Arranca desp del program code, y crece hacia abajo.
- <b>Stack</b>: Stack del proceso. Arranca en la ultima direccion del proceso y crece hacia arriba.

Esta es la gran abstraccion que hace el <b>sistema operativo</b> con cada programa. Ya que por supuesto ningun programa arranca en la direccion cero. Esta <b>abstraccion</b> es la conocida <b>virtualizacion de memoria</b>. Ya que el programa piensa que esta cargado en la direcion 0 pero en realidad no es asi.

Asi que cuando el programa carga algo en la direccion 0, el sistema operativo mediante un mecanismo de hardware mapeara a alguna direccion.

Esta <b>virtualizacion</b> es <b>invisible</b> al programa.

El mecanismo de virtualizacion debe cumplir estas tres caracteristicas:
- Debe ser <b>invisible</b> al programa.
- Debe ser <b>rapido</b>.
- Debe dar <b>proteccion</b>. Los <b>address space</b> son privados, cada proceso solo puede ver su address space. Este es el famoso <b>aislamiento</b>.

## Tipos de memoria

En un programa en ejecucion, existen dos tipos de memoria:
- <b>Stack</b>: Su reserve y liberacion es manejada implicitamente por el compilador, por eso a veces se la denomina <b>memoria automatica</b>.
- <b>Heap</b>: Este tipo de memoria es aquella que es obtenida y liberada explicitamente por el programador. Con es uso por ejemplo de <b>malloc</b> en C.

### Funciones para manejar memoria dinamica (heap): malloc & free

```c
int* numero = (int*) malloc(sizeof(int))
free(numero)
```

### Implementaciones

Veamos que <b>syscalls</b> son las que hacen posible la existencia de <b>malloc</b>. Bueno estas son:
- <b>brk()</b>: Lo que hace es cambiar el lugar donde le programa se rompe (hace break), es decir el lugar donde temrmina el <b>heap</b>. Solo recibe un parametro que es el lugar de la nueva direccion de fin del heap.
- <b>sbreak()</b>: Hace lo mismo que el break pero el parametro es la cantidad de bytes.

## Address Translation

Ahora vamos a entrar un poco mas en detalle de como se hace la traduccion de memoria. Esta es la tecnica que transforma una <b>direccion virtual</b> en una <b>direccion fisica</b>.

Todas las traducciones de memoria virtual a memoria fisica se hacen por hardware, esto es para que sean rapidas. La parte que se hace en hardware es digamos la mas basica, la operacion de traduccion pura, pero hay otras cosas que el que las hace es el sistema operativo.

A continuacion veremos las diferentes implementaciones que logran la virtualizacion de memoria.

### Dynamic Realocation o Memoria Segmentada

Esta es la primera implementacion, introducida hacia fin de los anios 50, es una idea simple llamada <b>base y segmento</b> tambien se le llama <b>dynamic realocation</b>.

En este implementacion mediante dos registros <b>base</b> y <b>segmento (tambien llamado registro limite)</b> se delimita la seccion de memoria fisica que perfenece al proceso. Estas dos variables son almacenadas en registros. Asi que el procesador para traducir de direccion <b>virtual</b> a <b>fisica</b> tiene que hacer lo siguiente:

\begin{equation}
Direccion Fisica = Base + Direccion Virtual
\end{equation}

Y ademas debe controlar que este proceso solo pueda acceder al address space de este proceso, esto lo hace fijandose en los registros <b>base y segmento</b>.

Hay dos variantes del registro <b>segmento</b> tambien llamado registro <b>bound</b>. Puede almacenar el tamanio de la seccion o bien puede almacenar la direccion del final de la seccion, cualquiera de estas dos variantes nos sirve para conocer el address space del programa.

### Address Translation con Tabla de Segmentos

Esta implementacion es parecida a la anterior pero tiene la diferencia de que en memoria fisica la porcion de memoria que mapea al address space del proceso no esta toda junta. Cada seccion mepea a un cacho diferente de memoria fisica. Entonces con dos registros ya no nos alcanza, ya que eso delimitaria un solo cacho de memoria, y nosotros necesitamos un cacho por cada seccion de memoria. Asi que necesitamos un par de registros por cada seccion, como son 4 secciones, stack, code, data, heap, necesitamos 2 x 4 registros.

<img src="Imagenes/address_translation_with_segment_table.jpg">

Cada direccion virtual se compone de dos partes:
- <b>Segmento</b>
- <b>Offset</b>

Con el <b>segmento</b> vamos a la tabla y agarramos la <b>base</b> y el <b>bound</b>, con esto ya conocemos el address space de esta seccion de este proceso. Y luego con el <b>offset</b> agarramos justo el dato que necesitamos de esta seccion. 

El error <b>Segmentation Fault</b> era un error que se daba cuando, en las maquinas que implementaban segmentacion, se queria direccionar a una direccion fuera de la seccion delimitada por base y bound. Increiblemente este error aun se utiliza en sistemas operativos que no usan segmentacion.

Una cosa a notar que no se dijo explicitamente es que cada proceso tiene su propia <b>Segment Table</b>.

La unidad de hardware que realiza la traduccion es la llamada <b>Memory Mangement Unit (MMU)</b>

## Memoria Paginada

En la memoria paginada la diferencia en que las direcciones en lugar de mapear a una seccion mapean a una pagina. Es decir que dividimos a la memoria fisica en paginas.

<img src="Imagenes/address_translation_with_page_table.jpg">

Cada direccion virtual se compone de dos partes:
- <b>Page Number</b>
- <b>Offset</b>

Con el <b>numero de pagina</b> vamos a la <b>page table</b> y obtenemos el <b>frame</b>, luego al frame le concatenamos el <b>offset</b> y logramos obtener la <b>direccion fisica</b>.

Una cosa a notar que no se dijo explicitamente es que cada proceso tiene su propia <b>Page Table</b>.

La unidad de hardware que realiza la traduccion es la llamada <b>Memory Mangement Unit (MMU)</b>

Una de las cosas que pueden parecer raras sobre la paginación es que si bien el programa cree que su memoria es lineal, de hecho, su memoria está desparramada por toda la memoria física como si fuera un mosaico.

## Paginación de memoria en x86

## Paged Segmentation ( Segmentación paginada)

Este sistema es una mezcla entre <b>paginacion</b> & <b>segmentacion</b>. Observar que tiene una <b>page table</b> y una <b>segment table</b>.

<img src="Imagenes/address_translation_with_paged_segmentation.jpg">

La <b>direccion virtual</b> aqui se compone de tres partes:
- <b>Segment</b>
- <b>Page</b>
- <b>Offset</b>

El funcionamiento es asi, tomamos de la direccion virtual el <b>segmento</b>, con eso vamos a la <b>segment table</b> esta nos va a devolver la <b>page table</b> y bueno obviamente ahora usamos el dato del <b>page number</b> que esta en la direccion virtual, esto nos da el <b>frame</b>, luego a esto le concatenamos el <b>offset</b> y finalmente obtenemos la <b>direccion fisica</b>.

## Address Translation con Tres Niveles de Page Tables

<img src="Imagenes/address_translation_with_three_level.jpg">

La <b>direccion virtual</b> aqui se compone de 3 partes:
- <b>Index 1</b>
- <b>Index 2</b>
- <b>Index 3</b>
- <b>Offset</b>

Agarramos el <b>index 1</b> y con eso indexamos en la tabla de paginas de nivel 1, que nos da la base de la tabla de paginas de nivel 2 y ahora usamos el <b>index 2</b> para obtener la base de la tabla de paginas de nivel 3, y aqui usamos el <b>index 3</b> donde finalmente agarramos el <b>frame</b> que le concatenamos el offset y obtenmos finalmente la <b>direccion fisica</b>.